In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
df = pd.read_csv("/Users/srinivasanc/Downloads/datasets/alzheimer.csv")

In [8]:
df.head()

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,Nondemented,M,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,Nondemented,M,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,Demented,M,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,Demented,M,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,Demented,M,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [10]:
df.isnull().sum()

Group     0
M/F       0
Age       0
EDUC      0
SES      19
MMSE      2
CDR       0
eTIV      0
nWBV      0
ASF       0
dtype: int64

In [14]:
data = df.dropna()

In [16]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [18]:
# Encode categorical data
le = LabelEncoder()
data['Group'] = le.fit_transform(data['Group'])  # Target variable
data['M/F'] = le.fit_transform(data['M/F'])      # Gender encoding

/var/folders/hl/jnm75lz146xfgkx412q2nv_c0000gn/T/ipykernel_64382/3844096131.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Group'] = le.fit_transform(data['Group'])  # Target variable
/var/folders/hl/jnm75lz146xfgkx412q2nv_c0000gn/T/ipykernel_64382/3844096131.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['M/F'] = le.fit_transform(data['M/F'])      # Gender encoding


In [20]:
# Features and target split
X = data.drop('Group', axis=1)
y = data['Group']

In [22]:
# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
# Handle class imbalance with SMOTE
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_scaled, y)

In [26]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [28]:
# Train a RandomForest model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [40]:
# Identify unique class labels AFTER SMOTE
unique_classes = np.unique(y_resampled)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred, target_names=['Demented', 'Nondemented', 'Converted']))

Accuracy: 0.91
              precision    recall  f1-score   support

    Demented       0.94      0.81      0.87        42
 Nondemented       0.89      1.00      0.94        40
   Converted       0.91      0.94      0.92        32

    accuracy                           0.91       114
   macro avg       0.91      0.92      0.91       114
weighted avg       0.92      0.91      0.91       114



In [42]:
# Sample new data (ensure it matches the original feature structure)
new_data = pd.DataFrame({
    'M/F': ['M'],          # Gender (encoded)
    'Age': [80],           # Age
    'EDUC': [12],          # Education level
    'SES': [2.0],          # Socioeconomic status
    'MMSE': [28.0],        # Mini-Mental State Examination
    'CDR': [0.5],          # Clinical Dementia Rating
    'eTIV': [1738],        # Estimated total intracranial volume
    'nWBV': [0.713],       # Normalized Whole Brain Volume
    'ASF': [1.010]         # Atlas Scaling Factor
})

# Encode categorical features
new_data['M/F'] = le.transform(new_data['M/F'])

# Scale the data using the previously fitted scaler
new_data_scaled = scaler.transform(new_data)

# Predict on the new data
prediction = model.predict(new_data_scaled)
predicted_label = ['Demented', 'Nondemented', 'Converted'][prediction[0]]  # Adjust based on your classes

print(f"Predicted Class: {predicted_label}")

Predicted Class: Nondemented
